In [15]:
import os
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
import missingno as msno
import sys
curPath = os.path.abspath(os.path.dirname('__file__'))
rootPath = os.path.split(curPath)[0]
sys.path.append(os.path.split(rootPath)[0])
# plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS'] # Mac下使用
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号


from loans_drop.config.path import data_path,config_path
from loans_drop.data_cleaning.decode_field import cat_replace
from loans_drop.config.feature import continues_columns
from loans_drop.feature_engineering.feature_select import missing_ratio, low_variance_filter, correlation

sys.path
def get_label(x):
    if x >= 0:
        return 1
    else:
        return 0

In [11]:
df = pd.read_csv(data_path + 'total_data20190724.csv',encoding = 'utf-8')
df['label']= df['product_amount'].apply(lambda x: get_label(x))
print(df['label'].value_counts())
print(df.shape)
print(df.info())
df.head()

0    69930
1     9506
Name: label, dtype: int64
(79436, 78)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79436 entries, 0 to 79435
Data columns (total 78 columns):
login_name                    79436 non-null int64
customer_id                   79436 non-null int64
product_amount                9506 non-null float64
follow_days                   18315 non-null float64
content                       72281 non-null object
salary_score                  72281 non-null float64
avgAddWechatFriendCount       78944 non-null float64
avgCallCount                  79188 non-null float64
avgIntentCustomerCount        63242 non-null float64
avgRemarkWords                79418 non-null float64
avgSignBillDays               43144 non-null float64
avgToDoorCount                73670 non-null float64
avgWorkHour                   79308 non-null float64
behaviorScore                 79408 non-null float64
loanRate                      78590 non-null float64
performanceRank               76525 non-nu

,login_name,customer_id,product_amount,follow_days,content,salary_score,avgAddWechatFriendCount,avgCallCount,avgIntentCustomerCount,avgRemarkWords,avgSignBillDays,avgToDoorCount,avgWorkHour,behaviorScore,loanRate,performanceRank,signBillCount,signBillRate,toDoorRate,age,locked,post,rank,sex,work_age,average_performance_amount,average_profit_amount,average_internal_cost,average_external_cost,average_payment_amount,average_business_amount,average_cost_price,average_CALL_0940_1040,average_CALL_1100_1200,average_CALL_1400_1500,average_CALL_1520_1620,average_achievement,average_profit,average_internal_cost_orf,average_external_cost_orf,average_assist_achievement,average_assist_profit,orderAmount,meetingCount,orderCount,tccs,flagContent,flagRemark,mon,remarkType,times,addTypeCode,allotNumber,businessLocation,businessOperate,businessStage,businessStatus,extensionCode,followTimes,invalidNumber,isCost,isMind,lastLoseType,opportunitytypeCode,originCode,originType,placeCode,referral,uploadType,vip,wayCode,widelyType,coverCharge,refundOrderSign,signOderNum,callTimes,avgCallDuration,label
0,104634,7793528449138606080,10500.0,NaN,NaN,NaN,0.0,49.0,0.46,47.4,NaN,1.46,9.75,102.4,0.0886,0.866485,38.0,0.0931,0.01,24.0,0.0,商务顾问,P,0.0,2.0,8284.86,7658.051,626.809,0.0,8328.86,13029.0,54.5,3.027027,2.513514,1.918919,2.162162,863.00625,797.713646,65.292604,0.0,0.0,0.0,36000.0,1.0,5.0,NaN,0.0,0.500000,NaN,NaN,NaN,BUS_ADD_TYPE_CODE_3,1.0,hid,BUS_704,NaN,BUS_STA_DELETE_CHECK,NaN,10.0,0.0,CBJSFS_NO,0.0,org,BUS_OPP_4,BUS_LYQD_707,ORIGIN_TYPE_4,BUS_SOR_PLACE_HZ,0.0,1.0,0.0,BUS_WAY_CODE_5,0.0,700.0,0.0,1.0,5.0,326.60,1
1,104634,7821553307750154240,12000.0,NaN,新增商机,0.0,0.0,49.0,0.46,47.4,NaN,1.46,9.75,102.4,0.0886,0.866485,38.0,0.0931,0.01,24.0,0.0,商务顾问,P,0.0,2.0,8284.86,7658.051,626.809,0.0,8328.86,13029.0,54.5,3.027027,2.513514,1.918919,2.162162,863.00625,797.713646,65.292604,0.0,0.0,0.0,12000.0,0.0,1.0,NaN,0.0,1.000000,2.0,NaN,NaN,BUS_ADD_TYPE_CODE_5,0.0,emp,BUS_3,NaN,BUS_STA_2_3,NaN,7.0,0.0,CBJSFS_NO,0.0,NaN,BUS_OPP_3,BUS_LYQD_101,ORIGIN_TYPE_6,BUS_SOR_PLACE_HZ,0.0,2.0,0.0,BUS_WAY_CODE_2,0.0,5958.6,0.0,0.0,NaN,NaN,1
2,104634,7797351044656316416,15000.0,NaN,移交商机给【金帅104634】，原因：无无,0.0,0.0,49.0,0.46,47.4,NaN,1.46,9.75,102.4,0.0886,0.866485,38.0,0.0931,0.01,24.0,0.0,商务顾问,P,0.0,2.0,8284.86,7658.051,626.809,0.0,8328.86,13029.0,54.5,3.027027,2.513514,1.918919,2.162162,863.00625,797.713646,65.292604,0.0,0.0,0.0,80000.0,0.0,3.0,NaN,0.0,1.000000,NaN,NaN,NaN,BUS_ADD_TYPE_CODE_5,0.0,hid,BUS_704,NaN,BUS_STA_DELETE_CHECK,NaN,2.0,0.0,CBJSFS_NO,0.0,org,BUS_OPP_3,BUS_LYQD_101,ORIGIN_TYPE_6,BUS_SOR_PLACE_HZ,0.0,1.0,0.0,BUS_WAY_CODE_3,0.0,7000.0,0.0,1.0,2.0,148.00,1
3,104634,7817560428952076288,2800.0,NaN,NaN,NaN,0.0,49.0,0.46,47.4,NaN,1.46,9.75,102.4,0.0886,0.866485,38.0,0.0931,0.01,24.0,0.0,商务顾问,P,0.0,2.0,8284.86,7658.051,626.809,0.0,8328.86,13029.0,54.5,3.027027,2.513514,1.918919,2.162162,863.00625,797.713646,65.292604,0.0,0.0,0.0,10300.0,0.0,2.0,NaN,0.0,0.888889,NaN,NaN,NaN,BUS_ADD_TYPE_CODE_11,2.0,emp,BUS_3,NaN,BUS_STA_2_3,NaN,25.0,0.0,JSCB,0.0,mng,BUS_OPP_1,BUS_LYQD_WT,ORIGIN_TYPE_10,BUS_SOR_PLACE_HZ,0.0,2.0,0.0,BUS_WAY_CODE_1,0.0,0.0,0.0,0.0,3.0,141.33,1
4,104634,7801663278289698816,39000.0,NaN,移交商机给【金帅104634】，原因：移交,0.0,0.0,49.0,0.46,47.4,NaN,1.46,9.75,102.4,0.0886,0.866485,38.0,0.0931,0.01,24.0,0.0,商务顾问,P,0.0,2.0,8284.86,7658.051,626.809,0.0,8328.86,13029.0,54.5,3.027027,2.513514,1.918919,2.162162,863.00625,797.713646,65.292604,0.0,0.0,0.0,45000.0,0.0,2.0,NaN,0.0,0.857143,NaN,NaN,NaN,BUS_ADD_TYPE_CODE_3,1.0,emp,BUS_3,NaN,BUS_STA_2_3,NaN,13.0,0.0,JSCB,0.0,NaN,BUS_OPP_1,BUS_LYQD_707,ORIGIN_TYPE_4,BUS_SOR_PLACE_HZ,0.0,2.0,0.0,BUS_WAY_CODE_3,0.0,0.0,0.0,0.0,8.0,162.63,1


In [12]:
# 相似度进行负采样，相似度太高的样本删除
def negativeSampling_with_followDays(k = 20, data=None, classifications='binary'):
    w = [1.0 * i / k for i in range(k + 1)]
    follow_series = data['follow_days'].describe(percentiles=w)[4:4 + k + 1]
    
    def experience_cut(x):
        if x >= follow_series['95%']:
            return 4
        elif x >= follow_series['75%']:
            return 3
        elif x >= follow_series['50%']:
            return 2
        elif x >= follow_series['25%']:
            return 1
        elif x is None:
            return 'null'
        else:
            return 0

    data['follow_bins']= data['follow_days'].apply(lambda x: experience_cut(x))
    data['follow_bins'].value_counts()
    
    df2 = data

    df1 = df2[df2['label'] == 0]
    print(df1.shape)
    
    df2 = df2[df2['label'] == 1]
    print(df2.shape)
    
    df3 = df1[(df1['follow_bins'] == 3) | (df1['follow_bins'] == 4)]
    print(df3.shape)
    
    if classifications == 'binary':
        df3['label'] = 1
    else:
        df3['label'] = 2
    
    test = df1[df1['follow_bins'] == 0]
    print(test.shape)
    df1 = df1[(df1['follow_bins'] == 1) | (df1['follow_bins'] == 2)]
    print(df1.shape)

    

    df1 = pd.concat([df1, df2, df3, test[:len(df1)]], axis=0)
    print(df1.shape)
    test = test[len(df1):]

    df1 = df1.sample(frac = 1.0, replace = True) #打乱顺序
    df1 = df1.reset_index().drop(columns=['index'])
    # df1.drop(columns=['login_name', 'customer_id'], inplace=True)
    print(df1.shape)
    
    df1 = df1[continues_columns + ['label', 'login_name', 'customer_id', 'follow_bins']]
    print(df1.shape)
    print(df1['label'].value_counts())
    df.dropna(axis=0, thresh=3, inplace=True)
    print(df1.shape)
    print(df1['label'].value_counts())
    
    test = test[df1.columns]
    
    return df1,test
    

In [13]:
df,test = negativeSampling_with_followDays(data=df, classifications='other')
print(test['follow_bins'].value_counts())

(69930, 79)
(9506, 79)
(4689, 79)
(55944, 79)
(9297, 79)


/Users/luocheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(32789, 79)
(32789, 79)
(32789, 51)
0    18513
1     9612
2     4664
Name: label, dtype: int64
(32789, 51)
0    18513
1     9612
2     4664
Name: label, dtype: int64
0    23155
Name: follow_bins, dtype: int64


In [14]:
# 测试(df1['follow_days1'] == 1) | (df1['follow_days1'] == 2)作为负样本是否合适
sample = df[['follow_bins', 'label']]
sample['count'] = 1
sample = sample.groupby(['follow_bins', 'label']).agg({'count':np.sum})
sample

/Users/luocheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count
follow_bins label       
0           0       9183
            1       9533
1           0       4735
            1         16
2           0       4595
            1          9
3           1         37
            2       3772
4           1         17
            2        892

In [8]:
remove = set()
    
print('处理连续型变量')
for name in continues_columns:
    if name in ['label', 'login_name', 'customer_id']:
        continue
    describe = df[name].describe()
    print(name, describe)
    print()
    if describe[4]==describe[6]:
        remove.add(name)
#         print(k,describe[4],describe[6])

ss = df.isnull().sum() / len(df)
for i in range(len(ss)):   
    if name == 'label':
        continue
    print(ss.index[i],ss[i])
    if ss[i] > 0.5:
        remove.add(ss.index[i])

remove = list(remove)
df.drop(columns = remove, axis = 1, inplace=True)
print(df.shape)
df.head()

处理连续型变量
avgAddWechatFriendCount count    32368.000000
mean         0.981034
std          6.826309
min          0.000000
25%          0.000000
50%          0.000000
75%          0.273000
max         94.174000
Name: avgAddWechatFriendCount, dtype: float64

avgCallCount count    32648.000000
mean        42.348291
std         15.437612
min          0.000000
25%         33.000000
50%         43.000000
75%         53.000000
max        162.000000
Name: avgCallCount, dtype: float64

avgIntentCustomerCount count    26929.000000
mean         0.405162
std          0.727933
min          0.040000
25%          0.090000
50%          0.170000
75%          0.330000
max          5.540000
Name: avgIntentCustomerCount, dtype: float64

avgRemarkWords count    32782.000000
mean        42.693414
std         18.037848
min          4.000000
25%         30.530000
50%         37.940000
75%         51.760000
max        156.230000
Name: avgRemarkWords, dtype: float64

avgSignBillDays count    18292.000000
mean    

avgAddWechatFriendCount 0.012839671841166245
avgCallCount 0.004300222635640001
avgIntentCustomerCount 0.17871847265851354
avgRemarkWords 0.0002134862301381561
avgSignBillDays 0.44212998261612124
avgToDoorCount 0.07645856842233675
avgWorkHour 0.0018298819726127666
behaviorScore 0.000762450821921986
loanRate 0.01314465216993504
performanceRank 0.03961694470706639
signBillCount 0.0029278111561804264
signBillRate 0.001311415413705816
toDoorRate 0.0062520967397602855
age 0.07069444020860655
work_age 0.07069444020860655
average_performance_amount 0.08109426941962243
average_profit_amount 0.08109426941962243
average_internal_cost 0.08109426941962243
average_external_cost 0.08109426941962243
average_payment_amount 0.08109426941962243
average_business_amount 0.08109426941962243
average_cost_price 0.08109426941962243
average_CALL_0940_1040 0.07078593430723719
average_CALL_1100_1200 0.07078593430723719
average_CALL_1400_1500 0.07078593430723719
average_CALL_1520_1620 0.07078593430723719
average_a

,avgAddWechatFriendCount,avgCallCount,avgIntentCustomerCount,avgRemarkWords,avgSignBillDays,avgToDoorCount,avgWorkHour,behaviorScore,loanRate,performanceRank,signBillCount,signBillRate,toDoorRate,age,work_age,average_performance_amount,average_profit_amount,average_internal_cost,average_external_cost,average_payment_amount,average_business_amount,average_cost_price,average_CALL_0940_1040,average_CALL_1100_1200,average_CALL_1400_1500,average_CALL_1520_1620,average_achievement,average_profit,average_internal_cost_orf,average_external_cost_orf,flagRemark,allotNumber,followTimes,callTimes,avgCallDuration,label,login_name,customer_id
0,0.000,5.0,NaN,29.94,15.0,NaN,8.17,100.0,0.6000,0.340599,5.0,0.0102,0.00,28.0,3.0,2620.571429,2328.307143,465.382143,76.964286,3562.160714,5648.892857,22.214286,1.757353,3.514706,3.845588,2.301471,2738.705816,2347.921633,137.741837,24.642857,0.900000,1.0,16.0,1.0,200.00,1,10824903,7807111893496676352
1,0.360,50.0,0.92,24.44,NaN,2.48,7.74,99.6,0.0759,0.114441,63.0,0.1987,0.59,24.0,8.0,5302.435714,4800.132500,516.767500,0.000000,5517.435714,16097.857143,100.500000,1.571429,1.435714,1.321429,1.071429,425.998596,387.472851,38.525746,0.826623,NaN,NaN,NaN,3.0,13.00,0,919232626,7752843941040627712
2,0.048,64.0,0.05,29.85,30.0,0.86,8.95,100.0,0.2727,0.803815,11.0,0.0259,0.48,29.0,9.0,7444.753784,6565.032162,879.721622,14.054054,7469.618649,2542.297297,49.864865,2.985294,6.036765,4.691176,3.764706,1954.942450,1711.834040,243.108411,5.347682,0.800000,1.0,4.0,13.0,143.31,1,905260842,1023115
3,0.000,63.0,0.35,54.71,NaN,1.17,9.65,100.0,0.1923,0.136240,25.0,0.1557,0.83,26.0,1.0,1042.000000,851.500000,190.500000,0.000000,1042.000000,3312.500000,26.250000,4.000000,3.350000,3.300000,3.850000,148.857143,121.642857,27.214286,0.000000,0.857143,1.0,4.0,7.0,344.71,1,105470,7825533505289240576
4,0.182,31.0,2.64,31.52,10.0,0.68,8.52,100.0,0.4038,0.754768,24.0,0.0410,0.03,25.0,26.0,10400.700215,7535.721720,2897.969355,259.274194,10901.736129,6247.369892,23.473118,2.894737,8.571429,4.586466,2.654135,5058.437565,3811.463217,946.426522,102.485507,0.846154,NaN,NaN,50.0,85.74,1,20435861,7755382778841276416


In [ ]:
df[['callTimes', 'followTimes', 'avgCallDuration', 'signBillRate']].corr()

# violinplot

In [ ]:
pal = sns.cubehelix_palette(2, rot=-.5, dark=.3)
for name in df.columns:
    sns.violinplot(data=df[[name]], palette=pal, inner="points")

# scatter_matrix

In [ ]:
# from pandas.tools.plotting import scatter_matrix
from pandas.plotting import scatter_matrix

fig,ax = plt.subplots(figsize=(10,10))
scatter_matrix(df[['avgWorkHour','signBillCount','signBillRate']],alpha = 0.2, diagonal='hist', ax=ax)

# kdeplot

In [ ]:
sns.kdeplot(df[['avgWorkHour','signBillCount']].dropna(how='any').values, shade=True, cut=0)


# 时间段统计分析

In [ ]:
sample['count'] = 1
sample = sample.groupby(['month', 'day']).agg({'count': np.sum}).reset_index()
print(sample.shape)
sample.head()

In [ ]:
flights = sample.pivot("month", "day", "count")
flights

In [ ]:
ax = sns.heatmap(flights)

# 散点图

In [ ]:
sns.jointplot(x="avgAddWechatFriendCount", y="avgCallCount", data=df[['avgAddWechatFriendCount', 'avgCallCount']]);

In [ ]:
sns.set(style="white", color_codes=True)

# Use JointGrid directly to draw a custom plot
grid = sns.JointGrid(df['avgAddWechatFriendCount'].values, df['avgCallCount'].values, space=0, height=6, ratio=50)
grid.plot_joint(plt.scatter, color="g")
grid.plot_marginals(sns.rugplot, height=1, color="g")

# Hexbin

In [ ]:
with sns.axes_style("white"):
    sns.jointplot(x=df['avgAddWechatFriendCount'].values, y=df['avgCallCount'].values, kind="hex", color="k");

# 构造新特征

In [ ]:
df['stage'] = df[['avgAddWechatFriendCount','avgCallCount']].mean(axis = 1)
df.head()

# 利用四分位数和标准差构造min max界限

In [ ]:
def make_report(dataframe):
    dic = {}
    for name in continues_columns:
        if name not in dataframe.columns:
            continue
            
        sample = dataframe[name].drop(dataframe[dataframe[name] == 0].index)
        dic2 = dict()
#         dic2['去除0后的总人数'] = len(sample)
        try:
            dic2['众数'] = sample.mode()[0]
        except:
            dic2['众数'] = None
        
        try:
            dic2['中位数'] = sample.median()
        except:
            dic2['中位数'] = None
        ss = sample.describe()
        for i in range(len(ss)):   
            dic2[ss.index[i]] = ss[i]

        value = dic2['75%'] + dic2['std']
        dic2['建议取值'] = value if dic2['max'] > value else dic2['max']
        print(name, dic2['75%'], dic2['std'], dic2['建议取值'], dic2['max'])
#         dic2['建议取值'] = dic2['建议取值'].astype(int)
        dic[name] = dic2
    return dic

df.describe()

In [ ]:
# df[['callTimes']].sort_values(by='callTimes', ascending=False)
df['callTimes'].value_counts()

In [ ]:
# df[['followTimes']].sort_values(by='followTimes', ascending=False)
df['followTimes'].value_counts()

In [ ]:
# df[['avgCallDuration']].sort_values(by='avgCallDuration', ascending=False)
df['avgCallDuration'].value_counts()

In [ ]:
# df[['signBillRate']].sort_values(by='signBillRate', ascending=False)
df['signBillRate'].value_counts()

In [ ]:
describe_df = pd.DataFrame(make_report(df))
describe_df

In [ ]:

df.head()

In [ ]:
from tqdm import tqdm_notebook as tqdm

for i in tqdm(range(len(df))):
    
    for name in df.columns:
        if name == 'label':
            continue
        limit = describe_df.loc['建议取值',name] 
        value = df.loc[i,name]
        if value > limit:
            random_low = describe_df.loc['25%',name]
            random_high = describe_df.loc['50%',name]
            random_value= np.random.uniform(low=random_low, high=random_high, size=1)[0]
            new_value = limit + random_value - random_low
            df.at[i,name] = new_value
#             print("特征{}: 建议取值:{} 实际值:{} 实际取值:{}".format(name, limit, value, new_value))
#             print()
        elif value < 0:
            df.at[i,name] = 0
    
            

# 模型训练

In [9]:
for name in df.columns:
    df[name].fillna(0, inplace=True)
    print(df[name].describe())

count    32789.000000
mean         0.968438
std          6.783242
min          0.000000
25%          0.000000
50%          0.000000
75%          0.269000
max         94.174000
Name: avgAddWechatFriendCount, dtype: float64
count    32789.000000
mean        42.166184
std         15.651645
min          0.000000
25%         32.000000
50%         43.000000
75%         53.000000
max        162.000000
Name: avgCallCount, dtype: float64
count    32789.000000
mean         0.332752
std          0.677701
min          0.000000
25%          0.050000
50%          0.120000
75%          0.320000
max          5.540000
Name: avgIntentCustomerCount, dtype: float64
count    32789.000000
mean        42.684299
std         18.046705
min          0.000000
25%         30.530000
50%         37.940000
75%         51.760000
max        156.230000
Name: avgRemarkWords, dtype: float64
count    32789.000000
mean        11.899905
std         12.487476
min          0.000000
25%          0.000000
50%         10.000000
7

In [10]:
from loans_drop.offline.lgb_train import LightGBM
model = LightGBM()
# model.split_data()
model.train(dataframe=df, save_path=config_path, classifications='other')

/Users/luocheng/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/Users/luocheng/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. 

train.label.value_counts: (22952, 38)
0    13097
1     6717
2     3138
Name: label, dtype: int64
test.label.value_counts: (9837, 38)
0    5612
1    2862
2    1363
Name: label, dtype: int64
数据拆分
训练集
验证集
测试集
Start training LightGBM...
[1]	valid_0's multi_logloss: 0.955913
[2]	valid_0's multi_logloss: 0.948797
[3]	valid_0's multi_logloss: 0.943138
[4]	valid_0's multi_logloss: 0.938416
[5]	valid_0's multi_logloss: 0.932856
[6]	valid_0's multi_logloss: 0.927547
[7]	valid_0's multi_logloss: 0.921034
[8]	valid_0's multi_logloss: 0.923297
[9]	valid_0's multi_logloss: 0.916737
[10]	valid_0's multi_logloss: 0.910439
[11]	valid_0's multi_logloss: 0.904088
[12]	valid_0's multi_logloss: 0.905931
[13]	valid_0's multi_logloss: 0.8998
[14]	valid_0's multi_logloss: 0.89543
[15]	valid_0's multi_logloss: 0.891298
[16]	valid_0's multi_logloss: 0.887152
[17]	valid_0's multi_logloss: 0.881796
[18]	valid_0's multi_logloss: 0.876658
[19]	valid_0's multi_logloss: 0.871053
[20]	valid_0's multi_logloss: 0.866343

[204]	valid_0's multi_logloss: 0.724853
[205]	valid_0's multi_logloss: 0.728673
[206]	valid_0's multi_logloss: 0.725241
[207]	valid_0's multi_logloss: 0.72687
[208]	valid_0's multi_logloss: 0.724204
[209]	valid_0's multi_logloss: 0.725899
[210]	valid_0's multi_logloss: 0.727428
[211]	valid_0's multi_logloss: 0.729057
[212]	valid_0's multi_logloss: 0.726182
[213]	valid_0's multi_logloss: 0.723706
[214]	valid_0's multi_logloss: 0.721029
[215]	valid_0's multi_logloss: 0.718851
[216]	valid_0's multi_logloss: 0.720583
[217]	valid_0's multi_logloss: 0.717987
[218]	valid_0's multi_logloss: 0.715411
[219]	valid_0's multi_logloss: 0.718468
[220]	valid_0's multi_logloss: 0.716136
[221]	valid_0's multi_logloss: 0.717834
[222]	valid_0's multi_logloss: 0.715102
[223]	valid_0's multi_logloss: 0.711804
[224]	valid_0's multi_logloss: 0.713585
[225]	valid_0's multi_logloss: 0.7155
[226]	valid_0's multi_logloss: 0.717027
[227]	valid_0's multi_logloss: 0.718186
[228]	valid_0's multi_logloss: 0.715932
[22

[412]	valid_0's multi_logloss: 0.590213
[413]	valid_0's multi_logloss: 0.591325
[414]	valid_0's multi_logloss: 0.589957
[415]	valid_0's multi_logloss: 0.588398
[416]	valid_0's multi_logloss: 0.586566
[417]	valid_0's multi_logloss: 0.587725
[418]	valid_0's multi_logloss: 0.58887
[419]	valid_0's multi_logloss: 0.587488
[420]	valid_0's multi_logloss: 0.588576
[421]	valid_0's multi_logloss: 0.586726
[422]	valid_0's multi_logloss: 0.584888
[423]	valid_0's multi_logloss: 0.583204
[424]	valid_0's multi_logloss: 0.581866
[425]	valid_0's multi_logloss: 0.583008
[426]	valid_0's multi_logloss: 0.581181
[427]	valid_0's multi_logloss: 0.579906
[428]	valid_0's multi_logloss: 0.581023
[429]	valid_0's multi_logloss: 0.579697
[430]	valid_0's multi_logloss: 0.581089
[431]	valid_0's multi_logloss: 0.579583
[432]	valid_0's multi_logloss: 0.578285
[433]	valid_0's multi_logloss: 0.576521
[434]	valid_0's multi_logloss: 0.577562
[435]	valid_0's multi_logloss: 0.578574
[436]	valid_0's multi_logloss: 0.577095
[

[621]	valid_0's multi_logloss: 0.521666
[622]	valid_0's multi_logloss: 0.522562
[623]	valid_0's multi_logloss: 0.521633
[624]	valid_0's multi_logloss: 0.522283
[625]	valid_0's multi_logloss: 0.521348
[626]	valid_0's multi_logloss: 0.520225
[627]	valid_0's multi_logloss: 0.519412
[628]	valid_0's multi_logloss: 0.518225
[629]	valid_0's multi_logloss: 0.518866
[630]	valid_0's multi_logloss: 0.519532
[631]	valid_0's multi_logloss: 0.518287
[632]	valid_0's multi_logloss: 0.517296
[633]	valid_0's multi_logloss: 0.517905
[634]	valid_0's multi_logloss: 0.518552
[635]	valid_0's multi_logloss: 0.517655
[636]	valid_0's multi_logloss: 0.516468
[637]	valid_0's multi_logloss: 0.515331
[638]	valid_0's multi_logloss: 0.515987
[639]	valid_0's multi_logloss: 0.516683
[640]	valid_0's multi_logloss: 0.515758
[641]	valid_0's multi_logloss: 0.516402
[642]	valid_0's multi_logloss: 0.516933
[643]	valid_0's multi_logloss: 0.515728
[644]	valid_0's multi_logloss: 0.516368
[645]	valid_0's multi_logloss: 0.515416


...

[1640]	valid_0's multi_logloss: 0.39768
[1641]	valid_0's multi_logloss: 0.397844
[1642]	valid_0's multi_logloss: 0.397942
[1643]	valid_0's multi_logloss: 0.398107
[1644]	valid_0's multi_logloss: 0.398204
[1645]	valid_0's multi_logloss: 0.398344
[1646]	valid_0's multi_logloss: 0.398473
[1647]	valid_0's multi_logloss: 0.398603
[1648]	valid_0's multi_logloss: 0.398327
[1649]	valid_0's multi_logloss: 0.398479
[1650]	valid_0's multi_logloss: 0.398606
[1651]	valid_0's multi_logloss: 0.39833
[1652]	valid_0's multi_logloss: 0.398473
[1653]	valid_0's multi_logloss: 0.398183
[1654]	valid_0's multi_logloss: 0.397903
[1655]	valid_0's multi_logloss: 0.398058
[1656]	valid_0's multi_logloss: 0.397763
[1657]	valid_0's multi_logloss: 0.397494
[1658]	valid_0's multi_logloss: 0.397235
[1659]	valid_0's multi_logloss: 0.396986
[1660]	valid_0's multi_logloss: 0.397103
[1661]	valid_0's multi_logloss: 0.397247
[1662]	valid_0's multi_logloss: 0.397002
[1663]	valid_0's multi_logloss: 0.396774
[1664]	valid_0's m

[1842]	valid_0's multi_logloss: 0.385926
[1843]	valid_0's multi_logloss: 0.386034
[1844]	valid_0's multi_logloss: 0.386158
[1845]	valid_0's multi_logloss: 0.38626
[1846]	valid_0's multi_logloss: 0.386078
[1847]	valid_0's multi_logloss: 0.386181
[1848]	valid_0's multi_logloss: 0.386004
[1849]	valid_0's multi_logloss: 0.385808
[1850]	valid_0's multi_logloss: 0.385567
[1851]	valid_0's multi_logloss: 0.385648
[1852]	valid_0's multi_logloss: 0.385727
[1853]	valid_0's multi_logloss: 0.385824
[1854]	valid_0's multi_logloss: 0.38563
[1855]	valid_0's multi_logloss: 0.385746
[1856]	valid_0's multi_logloss: 0.385885
[1857]	valid_0's multi_logloss: 0.385706
[1858]	valid_0's multi_logloss: 0.385834
[1859]	valid_0's multi_logloss: 0.385674
[1860]	valid_0's multi_logloss: 0.385753
[1861]	valid_0's multi_logloss: 0.385585
[1862]	valid_0's multi_logloss: 0.385668
[1863]	valid_0's multi_logloss: 0.385435
[1864]	valid_0's multi_logloss: 0.385263
[1865]	valid_0's multi_logloss: 0.385064
[1866]	valid_0's m

/Users/luocheng/Documents/dgg/bss-recommended-platform/loans_drop/offline/lgb_train.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['probability'] = [','.join([str(i) for i in pred_list]) for pred_list in preds]


accuracy_score: 0.8128494459692995

              precision    recall  f1-score   support

           0       0.78      0.95      0.86      5612
           1       0.91      0.85      0.88      2862
           2       0.72      0.17      0.28      1363

    accuracy                           0.81      9837
   macro avg       0.80      0.66      0.67      9837
weighted avg       0.81      0.81      0.78      9837



/Users/luocheng/Documents/dgg/bss-recommended-platform/loans_drop/offline/lgb_train.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['label_pred'] = preds


In [ ]:
model.feature_evaluation()

In [ ]:
preds = model.model.predict(test[df.columns].drop(['label'], axis=1), num_iteration=model.model.best_iteration)
Y = test['follow_bins'].values.tolist()
for i in range(len(preds)):
    if preds[i] > 0.5:
        print(preds[i], Y[i])
    if Y[i] >= 3:
        print(preds[i], Y[i])

In [ ]:
preds = [1 if i > 0.5 else 0 for i in preds]
index = [preds[i] != Y[i] for i in range(len(preds))]
test['pred_label'] = preds
test[index].head()

In [ ]:
preds = model.model.predict(test[df.columns].drop(['label'], axis=1), num_iteration=model.model.best_iteration)
Y = test['follow_bins'].values.tolist()
for i in range(len(preds)):
    print(np.argmax(preds[i]), Y[i])
    if np.argmax(preds[i]) == 2:
        print('被预测为不容易调库')

In [ ]:
','.join([0.2, 0.3, 0.4])

In [ ]:
print(df.shape)
df.dropna(inplace=True)
print(df.shape)
for name in df.columns:
    MIDSIZE = (12, 8)
    fig, ax = plt.subplots(figsize=MIDSIZE)
    df[name].value_counts(dropna=False, ascending=True).plot(kind='barh', ax=ax)
    ax.set_ylabel(name)
    ax.set_xlabel("Counts")
    fig.tight_layout()

In [ ]:
sns.catplot(y="follow_bins", hue="label", kind="count", palette="pastel", edgecolor=".6", data=df)

In [ ]:
df['']